# Prosper Loan Dataset : L'objectif est de prédire le défaut de remboursement d’un prêt.

### On commence avec l'importation de libraries

In [62]:
import pandas as pd

### On importe le dataset

In [63]:
df = pd.read_csv("prosperLoanData.csv")

#### Ce jeu de données provient de la société de prêts Prosper. Il regroupe 113 937 prêts décrits par 81 variables

In [64]:
df.shape

(113937, 81)

In [65]:
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


## Defition de la variable cible : la variablale cible est "LoanStatus", qui indique le statut du prêt.


In [66]:
df["LoanStatus"].unique()

array(['Completed', 'Current', 'Past Due (1-15 days)', 'Defaulted',
       'Chargedoff', 'Past Due (16-30 days)', 'Cancelled',
       'Past Due (61-90 days)', 'Past Due (31-60 days)',
       'Past Due (91-120 days)', 'FinalPaymentInProgress',
       'Past Due (>120 days)'], dtype=object)

### On va se concentrer sur les prêts sains et les prêts en défaut

 On crée la variable **LoanStatusBinaire** afin de définir la variable cible au format binaire, qui sera celle à prédire lors de la modélisation

In [67]:
# On essayera de prédire si un prêt sera remboursé ou non.
df = df[df["LoanStatus"].isin(['Completed', 'Current', 'Defaulted',
       'Chargedoff', 'Past Due (61-90 days)', 'Past Due (31-60 days)',
       'Past Due (91-120 days)', 'FinalPaymentInProgress',
       'Past Due (>120 days)'])]

In [68]:
df["LoanStatusBinaire"] = df["LoanStatus"].apply(lambda x: 0 if x in ['Completed', 'Current','FinalPaymentInProgress'] else 1)

## Exploration des valeurs manquantes

Nous calculons la proportion de valeurs manquantes pour chaque colonne. Cela guidera nos décisions :


- Les colonnes présentant un taux de valeurs manquantes très élevé ( > 20 %) ne sont généralement pas utiles pour la modélisation et peuvent introduire un biais dans les résultats.

In [69]:
# Pourcentage de valeurs manquantes par colonne
missing_ratio = df.isna().mean().sort_values(ascending=False)

# Afficher les colonnes ayant le plus de valeurs manquantes
missing_ratio

GroupKey                         0.882014
LoanFirstDefaultedCycleNumber    0.849824
ScorexChangeAtTimeOfListing      0.834947
ProsperPrincipalBorrowed         0.807179
TotalProsperLoans                0.807179
                                   ...   
Recommendations                  0.000000
InvestmentFromFriendsCount       0.000000
InvestmentFromFriendsAmount      0.000000
Investors                        0.000000
LoanStatusBinaire                0.000000
Length: 82, dtype: float64

#### Supprimer les colonnes avec un taux de valeurs manquantes très élevé (> 20%)


In [70]:
missing_procentage= 0.2
high_missing_cols = missing_ratio[missing_ratio > missing_procentage].index
print(f"Columns with > {missing_procentage*100:.0f}% missing values:")
print(high_missing_cols)

# Supprimer les colonnes avec un taux de valeurs manquantes très élevé
df = df.drop(columns=high_missing_cols)
print(f"Shape after dropping high-missing columns: {df.shape}")

Columns with > 20% missing values:
Index(['GroupKey', 'LoanFirstDefaultedCycleNumber',
       'ScorexChangeAtTimeOfListing', 'ProsperPrincipalBorrowed',
       'TotalProsperLoans', 'TotalProsperPaymentsBilled',
       'OnTimeProsperPayments', 'ProsperPaymentsOneMonthPlusLate',
       'ProsperPrincipalOutstanding', 'ProsperPaymentsLessThanOneMonthLate',
       'CreditGrade', 'ClosedDate', 'ProsperRating (numeric)',
       'EstimatedReturn', 'EstimatedLoss', 'EstimatedEffectiveYield',
       'ProsperScore', 'ProsperRating (Alpha)'],
      dtype='object')
Shape after dropping high-missing columns: (112861, 64)


## Suppression des identifiants ou colonnes techniques évidents

Certaines variables sont des identifiants ou des clés (par exemple, des identifiants internes). Elles ne contiennent pas d'information pertinente pour la prédiction de la cible et peuvent aussi introduire des fuites d'information.

In [71]:
df.columns

Index(['ListingKey', 'ListingNumber', 'ListingCreationDate', 'Term',
       'LoanStatus', 'BorrowerAPR', 'BorrowerRate', 'LenderYield',
       'ListingCategory (numeric)', 'BorrowerState', 'Occupation',
       'EmploymentStatus', 'EmploymentStatusDuration', 'IsBorrowerHomeowner',
       'CurrentlyInGroup', 'DateCreditPulled', 'CreditScoreRangeLower',
       'CreditScoreRangeUpper', 'FirstRecordedCreditLine',
       'CurrentCreditLines', 'OpenCreditLines', 'TotalCreditLinespast7years',
       'OpenRevolvingAccounts', 'OpenRevolvingMonthlyPayment',
       'InquiriesLast6Months', 'TotalInquiries', 'CurrentDelinquencies',
       'AmountDelinquent', 'DelinquenciesLast7Years',
       'PublicRecordsLast10Years', 'PublicRecordsLast12Months',
       'RevolvingCreditBalance', 'BankcardUtilization',
       'AvailableBankcardCredit', 'TotalTrades',
       'TradesNeverDelinquent (percentage)', 'TradesOpenedLast6Months',
       'DebtToIncomeRatio', 'IncomeRange', 'IncomeVerifiable',
       'StatedMo

In [72]:
id_columns = ['ListingKey', 'ListingNumber', 'LoanKey', 'MemberKey']
df = df.drop(columns=id_columns)

In [73]:
df.shape

(112861, 60)

## Transformation des variables dates

On peut constater que les variables contenant des dates ne sont pas dans le bon format : elles sont de type object. Il est donc nécessaire de les convertir en type date

In [74]:
list_dates = ['ListingCreationDate', 'LoanOriginationDate']
for col in list_dates:
    print("La variable ", col, "a pour format :", df[col].dtype)

La variable  ListingCreationDate a pour format : object
La variable  LoanOriginationDate a pour format : object


In [75]:
for col in list_dates:
    df[col]= pd.to_datetime(df[col], errors='coerce')

La variable DateCreditPulled présentait des formats de dates mixtes ; on a donc nettoyé les micro/nanosecondes pour permettre à Pandas de la convertir correctement en datetime

In [78]:
df["DateCreditPulled"] = (
    df["DateCreditPulled"]
    .astype(str)
    .str.replace(r"\.\d{6,9}$", "", regex=True)  # remove micro/nanoseconds
)

df["DateCreditPulled"] = pd.to_datetime(
    df["DateCreditPulled"], 
    errors="coerce"
)

In [84]:
df["DateCreditPulled"].head()

0   2007-08-26 18:41:46
1   2014-02-27 08:28:14
2   2007-01-02 14:09:10
3   2012-10-22 11:02:32
4   2013-09-14 18:38:44
Name: DateCreditPulled, dtype: datetime64[ns]

On a converti la variable **LoanOriginationQuarter** en une vraie date en utilisant le début de chaque trimestre, afin de pouvoir l’utiliser correctement dans les analyses temporelles. On a séparé le trimestre et l’année pour reformer un format que Pandas peut comprendre (‘AAAAQx’), puis on  l’a converti en date représentant le début de chaque trimestr

In [ ]:
tmp = df["LoanOriginationQuarter"].str.split(" ", expand=True)

df["LoanOriginationQuarter"] = (
    tmp[1] + "Q" + tmp[0].str[1]
).astype("period[Q]").dt.to_timestamp()

# Analyze Exploratoire

 # Feature Engineering

### On selectionne les principales colonnes d'intérêt pour la modelisation

- **CreditGrade** : The credit rating that was assigned at the time the listing went live. Applicable for listings pre-2009 period and will only be populated for those listings.
- **Term** : The length of the loan expressed in months.
- **BorrowerAPR** : The borrower's Annual Percentage Rate (APR) for the loan. **(Pour la modélisation, le BorrowerAPR est plus pertinent que le BorrowerRate, car il reflète le coût réel du crédit en intégrant les frais, offrant ainsi une mesure plus complète et comparable du risque d’emprunt.)**
- **ProsperRating (numeric)** : The Prosper Rating assigned at the time the listing was created:  
  `0 = N/A, 1 = HR, 2 = E, 3 = D, 4 = C, 5 = B, 6 = A, 7 = AA.`  

- **ProsperScore** : A custom risk score built using historical Prosper data. Score ranges from 1–10, with **10 being the best** (lowest risk). Applicable for loans originated after July 2009.

- **ListingCategory** : The category of the listing selected by the borrower when posting their listing:  
  `0 - Not Available, 1 - Debt Consolidation, 2 - Home Improvement, 3 - Business, 4 - Personal Loan, 5 - Student Use, 6 - Auto, 7 - Other, 8 - Baby&Adoption, 9 - Boat, 10 - Cosmetic Procedure, 11 - Engagement Ring, 12 - Green Loans, 13 - Household Expenses, 14 - Large Purchases, 15 - Medical/Dental, 16 - Motorcycle, 17 - RV, 18 - Taxes, 19 - Vacation, 20 - Wedding Loans`

  - **BorrowerState** : The two-letter abbreviation of the borrower’s state at the time the listing was created.

- **Occupation** : The occupation selected by the borrower at the time they created the listing.

- **EmploymentStatus** : The employment status of the borrower at the time they posted the listing.

- **EmploymentStatusDuration** : The length (in months) of the employment status at the time the listing was created.

- **IsBorrowerHomeowner** : Indicates whether the borrower is a homeowner (based on mortgage or documentation confirming homeownership).
- **CurrentCreditLines** : Number of current credit lines at the time the credit profile was pulled.




